## MicroPython ESP32 Experimentation

### Establishing connection to target board
First, make sure you've got the right serial port. On unix-based systems, you can run `ls /dev/tty.*` to see your available serial devices. Replace as necessary below.

This will allow Jupyter (your host computer) to run commands and send/receive information to/from your target board in real time using the MicroPython REPL.

In [121]:
%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
# %serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-02U1W54L --baud=115200 
MicroPython cb99ca9-dirty on 2021-11-01; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [120]:
%sendtofile lib/logging.py --source lib/logging.py

Sent 18 lines (415 bytes) to lib/logging.py.


In [113]:
%sendtofile lib/runner.py --source lib/runner.py

Sent 212 lines (6613 bytes) to lib/runner.py.


In [130]:
from scheduling import ScheduledFunc

class RunnerLogger(ScheduledFunc):

    def __init__(self, period_sec, output_buffer_ref, timer_num=1):
        super().__init__(timer_num, 1/period_sec)
        self.buffer = output_buffer_ref
        self.setup()

    def setup(self):
        pass

    def log(self, *args):
        print(self.buffer)

    def start(self):
        self.tim.init(freq=self.freq, callback=self.log)

In [131]:
class Run():
    def __init__(self):
        self.buffer = [0.5 for i in range(3)]
        self.logger = RunnerLogger(2, self.buffer)
        
class Log():
    def __init__(self, buff):
        self.buff = buff
        
    def log(self):
        print(self.buff)

In [132]:
runner = Run()

In [133]:
runner.logger.start()

In [134]:
runner.logger.stop()

[leftinbuffer] ['[0.5, 0.5, 0.5]']
[leftinbuffer] ['[0.5, 0.5, 0.5]']
[leftinbuffer] ['[0.5, 0.5, 0.5]']
[leftinbuffer] ['[0.5, 0.5, 0.5]']


In [99]:
runner.logger.log()

[0.5, 0.5, 2]


In [71]:
from lib.runner import Runner, BaseRunner

In [72]:
runner = BaseRunner()

In [73]:
runner.setup()

ADC initialised
SPI initialised
DigiPot set to minimum gain (1.8)


In [82]:
print(runner.decode())

{12: 0.14172, 10: 0.11683, 7: 0.06797}


In [79]:
print(runner.read_output_buffer())

[0.01954444, 4.355747, 15.51876, -8.554688, -31.78373, 6.855578, 3.560656, -3.856112, 12.3577, -6.898592, 6.194204, 4.138245, 3.281769, 8.534527, -1.336413, 4.062249, -5.361069, 0.3286598, -4.770914, -8.813795, -5.314931, -4.990621, 3.239691, -1.65804, 2.832161, 2.307831, 1.354408, 2.087882, 1.007222, 0.4961143, -2.649651, 0.1243045, 0.7833976, 3.787482, 2.706089, 3.938741, 2.410984, 0.9986733, 0.6800158, -4.813919, -3.0345, -6.011649, -3.914723, 0.4714172, 1.783925, 2.19733, 0.7634152, 2.212833, -0.3497142, -0.4576373, -2.63616, -4.878806, -3.71775, -0.4853296, 2.104439, 1.520316, 5.232512, 4.439618, 2.400053, 2.780979, 0.711254, -2.359647, -4.36913, -1.930319, -0.1143028, 0.3615065, 8.597612, 6.993036, -16.48263, -12.89078, 3.261904, -2.795664, 5.108867, 8.50136, -3.086584, 7.453905, 6.905039, -7.932702, 6.488844, -8.103814, -10.15893, 10.35165, -4.385473, 10.4655, -5.383912, -4.192613, 13.19026, -17.59774, 3.503542, -4.240097, -12.53785, 11.49746, 0.7746119, 10.87862, -0.7757437, -4

In [ ]:
from lib.requests import MicroWebCli as requests

requests.GETRequest("https://google.com")

### Testing your WiFi connection
In order to connect to a local WiFi network, you'll need to supply your network SSID and password in a `.env` file on the board. Doing this is easy: 
1. On your computer, create a `.env` file using `touch .env`. Update the `.env` file with the required fields:
    
    ```bash
    #.env 
    WIFI_SSID=<your network name>
    WIFI_PASSWORD=<your network password>
    
    ```
    
2. Send this file to your target device using the following command:
    ```ipython
%sendtofile --source lib/.env lib/.env  --binary
```

You may need to update the local (source) path to your `.env` file depending on where you created/stored it.

In [135]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('192.168.0.28', '255.255.255.0', '192.168.0.1', '192.168.0.1')


In [58]:
from lib.core import initialise, run
import machine

adc_params = {
    'adc_pin': 33, # as used in the Imperial EEG hardware prototype
    'atten': machine.ADC.ATTN_11DB, # see ESP32 docs for options
    'width': machine.ADC.WIDTH_12BIT, # precision (12-bit is max for ESP32)
    'buffer_size': 256, # semi-arb. Aligned with the 256Hz sample freq for 1 second of data in mem.
}

# use garbage collector liberally to save memory where possible
# but try not call `gc.collect` to often after creating/destroying 
# small objects as this will cause fragmentation
gc.collect()
periph_manager, mqtt_client, data_scheduler = initialise(offline=False, adc_params=adc_params)
gc.collect()

ADC initialised
SPI initialised
DigiPot set to minimum gain (1.8)
connecting to network...
network config: ('192.168.0.22', '255.255.255.0', '192.168.0.1', '192.168.0.1')
Attemptint to connect to socket addr:  ('3.10.203.127', 1883)
resp:  b' \x02\x00\x00'
0x4 b'32:46:00:00'


In [3]:
gc.collect()
run() 

In [59]:

### NB!!! Make sure filter coefficients match the correct samplign frequency!

def sos_filter(sig, fs=None):
    from ulab import numpy as np
    from ulab import scipy as spy
    sos_coeffs = np.array([[ 5.18206655e-04,  5.90798154e-04,  5.18206655e-04,
         1.00000000e+00, -1.58702496e+00,  6.47839164e-01],
       [ 1.00000000e+00, -6.71623649e-01,  1.00000000e+00,
         1.00000000e+00, -1.56165946e+00,  7.42958422e-01],
       [ 1.00000000e+00, -1.19857302e+00,  1.00000000e+00,
         1.00000000e+00, -1.53434838e+00,  8.53019872e-01],
       [ 1.00000000e+00, -1.36458617e+00,  1.00000000e+00,
         1.00000000e+00, -1.52074631e+00,  9.31081209e-01],
       [ 1.00000000e+00, -1.41818431e+00,  1.00000000e+00,
         1.00000000e+00, -1.52570486e+00,  9.80262684e-01]])
    return spy.signal.sosfilt(sos_coeffs, sig)

def preprocess_data(signal, downsample_freq=None):
    from ulab import numpy as np
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = downsample_freq or DOWNSAMPLED_FREQ
    ds_factor = ADC_SAMPLE_FREQ//downsample_freq
    signal = np.array(signal) - np.mean(signal) # remove DC component
    return sos_filter(signal, fs=ADC_SAMPLE_FREQ)[::ds_factor] # NOTE: need to supply correct sampling freq

def web_log_callback(*args, **kwargs):
    from lib.requests import MicroWebCli as requests

    global periph_manager
    global output_buffer
    global sample_counter
    global log_tim
    
    log_tim.deinit()
    
    periph_manager.write_led("green", 1)
    packed_data = {'data': output_buffer, 'timestamp': time.ticks_us(), 'session_id': DEFAULT_LOG_SESSION}
    requests.POSTRequest('http://james-tev.local:5000/', packed_data)
        
    sample_counter = 0
    output_buffer = [0.0 for i in range(256)]
    periph_manager.write_led("green", 0)
    
    # restart log timer
    log_tim.init(freq=log_freq, callback=web_log_callback)
    
def sample_callback(*args, **kwargs):
    from lib.utils import update_buffer 
    
    global periph_manager
    global sample_counter
    global output_buffer
    
    periph_manager.adc_read_to_buff(size=1)
    sample_counter += 1
    
    # this will only be true every 1s once buffer fills
    if sample_counter == periph_manager._adc_params['buffer_size']:
        periph_manager.write_led('red', 1)
        data = periph_manager.read_adc_buffer()
        data = preprocess_data(data, downsample_freq = DOWNSAMPLED_FREQ)
        output_buffer = update_buffer(output_buffer, list(data), 256)
        sample_counter = 0
        periph_manager.write_led('red', 0)


In [61]:
from machine import Pin, Timer
from lib.networking import pack_payload
import utime as time

ADC_SAMPLE_FREQ = 256
RECORDING_LEN_SEC = 4
OVERLAP = 0.5
DOWNSAMPLED_FREQ = 64 # 64 Hz downsampled  to ensure nyquist condition
DEFAULT_LOG_SESSION = 'test-{0}'.format(time.ticks_ms())

topic = 'james_esp32'
sample_counter = 0
output_buffer = [0.0 for i in range(256)]

# start sampling on timer 0
adc_tim = Timer(0)
adc_tim.init(freq=ADC_SAMPLE_FREQ, callback=sample_callback)

# flash LED
periph_manager.write_led("green", 1)

time.sleep(RECORDING_LEN_SEC) # wait for at least one full window

# init web logging 
periph_manager.write_led("green", 0)
buff_size = 256
log_freq = (DOWNSAMPLED_FREQ/buff_size)*0.75
log_tim = Timer(1)
log_tim.init(freq=log_freq, callback=web_log_callback)

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    log_tim.deinit()
    adc_tim.deinit()
    gc.collect()
    print("received keyboard interrupt")

.......Traceback (most recent call last):
  File "<stdin>", line 46, in web_log_callback
  File "lib/requests.py", line 125, in POSTRequest
  File "lib/requests.py", line 325, in OpenRequestFormData
  File "lib/requests.py", line 72, in _quote
MemoryError: memory allocation failed, allocating 3318 bytes
....................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: [Errno 6] Device not configured


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: [Errno 6] Device not configured



In [6]:
from machine import Pin, Timer

# flash LED
STIM_FREQ = 7
led = Pin(27, Pin.OUT)

def led_callback(*args, **kwargs):
    led.value(not led.value())

led_tim = Timer(2)
led_tim.init(freq=STIM_FREQ*2, callback=led_callback)

In [10]:
led_tim.deinit()

In [3]:
import utime as time
from machine import Pin, Timer
from lib.decoding import CCA
from lib.networking import pack_payload

global periph_manager
global mqtt_client
global topic

STIM_FREQ = 12
PREPROCESSING = False # if true, LP filter and downsample

cca_ref_freq = ADC_SAMPLE_FREQ
if PREPROCESSING:
    cca_ref_freq = DOWNSAMPLED_FREQ
    
stim_freqs = [7, 10, 12]
cca_decoder = CCA(stim_freqs, cca_ref_freq) # !! use DOWNSAMPLED_FREQ for harmonic reference if using downsampling on original signal



In [22]:
from lib.requests import MicroWebCli as requests
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars(".env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

contentBytes = requests.POSTRequest('http://james-tev.local:5000/', {'data': [1,2,3], 'session_id': 'session-1'})
print(contentBytes)

network config: ('192.168.0.26', '255.255.255.0', '192.168.0.1', '192.168.0.1')
b'data stored successfully'


In [9]:
# flash LED
led = Pin(26, Pin.OUT)

def led_callback(*args, **kwargs):
    led.value(not led.value())

led_tim = Timer(2)
led_tim.init(freq=STIM_FREQ*2, callback=led_callback)

topic = "james_esp32"

time.sleep(RECORDING_LEN_SEC) # wait for at least one full window
while True:
    time.sleep(RECORDING_LEN_SEC*(1-OVERLAP))
    decoded_data = read_and_decode(preprocessing=PREPROCESSING)
    gc.collect()
    print(decoded_data)
#     payload = pack_payload([1,2,3], decoded_data, client_id=mqtt_client.client_id)
    gc.collect()
#     mqtt_client.publish(topic=topic, msg=payload, qos=1)

{12: 0.0417, 10: 0.0691, 7: 0.0475}
.{12: 0.0495, 10: 0.0457, 7: 0.07280001}
{12: 0.0355, 10: 0.0505, 7: 0.0236}
{12: 0.0713, 10: 0.0587, 7: 0.0683}
.{12: 0.0673, 10: 0.0296, 7: 0.0563}
{12: 0.08990001, 10: 0.0497, 7: 0.0622}
{12: 0.0514, 10: 0.0837, 7: 0.0742}
.{12: 0.0611, 10: 0.0327, 7: 0.0702}
{12: 0.0955, 10: 0.0594, 7: 0.0512}
{12: 0.0439, 10: 0.0543, 7: 0.07150001}
{12: 0.0341, 10: 0.0359, 7: 0.0226}
.{12: 0.5333, 10: 0.0814, 7: 0.0906}
{12: 0.6369, 10: 0.0346, 7: 0.0002}
{12: 0.6369, 10: 0.0347, 7: 0.0001}
.{12: 0.6369, 10: 0.0346, 7: 0.0001}
{12: 0.6369, 10: 0.0346, 7: 0.0001}
{12: 0.6369, 10: 0.0345, 7: 0.0001}
.{12: 0.6369, 10: 0.0347, 7: 0.0001}
{12: 0.4701, 10: 0.1064, 7: 0.1063}
{12: 0.0273, 10: 0.024, 7: 0.024}
{12: 0.0832, 10: 0.0489, 7: 0.08160001}
.{12: 0.0576, 10: 0.0376, 7: 0.0654}
{12: 0.0618, 10: 0.0305, 7: 0.0446}

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: [Errno 6] Device not configured


**[ys] <class 'serial.serialutil.SerialExcept

In [ ]:
import urandom

def synth_x(f, Ns, noise_power=0.5, fs=250):
    """
    generate a synthetic signal vector
    
    args:
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    """
    t = np.arange(0, Ns/fs, 1/fs)
    return np.sin(t*2*np.pi*f)*(1+urandom.random()*noise_power)

def synth_X(f, Nc, Ns, noise_power=0.5, fs=200, f_std=0.02, noise_std=0.2):
    """
    Generate a matrix of several variations of the same target signal. This is used
    to simulate the measurement of a common signal over multiple EEG channels 
    that have different SNR characteristics.
    
    args:
    f [float]: target frequency of synthetic signal (Hz)
    Nc [int]: number of channels
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    fs [float]: sampling frequency (Hz)
    f_std [float]: standard dev. of freq. in generated signal across channels to simulate interference from other frequency components over different channels
    noise_std [float]: standard dev. of noise across channels
    """
    X = []
    for i in range(Nc): # simulate noisy sinusoids with varying SNR across Nc channels
        f_i = f*(1+urandom.random()*0.1)
        x = synth_x(f_i, Ns, noise_power=0)

        X.append(x)
        
    return np.array(X)

def harmonic_reference(f0, fs, Ns, Nh=2, standardise_out=False):
    
    '''
    Generate reference signals for canonical correlation analysis (CCA)
    -based steady-state visual evoked potentials (SSVEPs) detection [1, 2].
    function [ y_ref ] = cca_reference(listFreq, fs,  Ns, Nh) 
    Input:
      f0        : stimulus frequency
      fs              : Sampling frequency
      Ns              : # of samples in trial
      Nh          : # of harmonics
    Output:
      y_ref           : Generated reference signals with shape (Nf, Ns, 2*Nh)
    '''  

    tidx = np.arange(1,Ns+1)*(1/fs) #time index
    
    tmp = []
    for harm_i in range(1,Nh+1):
        # Sin and Cos
        s = np.sin(tidx*2*np.pi*harm_i*f0)
        tmp.extend([s,
                    np.cos(tidx*2*np.pi*harm_i*f0)])
    y_ref = np.array(tmp)
    if standardise_out: # zero mean, unit std. dev
        return standardise(y_ref)
    return y_ref

In [ ]:
import gc

gc.collect()

Ns = 50

X = synth_X(7, 4, Ns, noise_power=0.2, f_std=0.04)
Y = harmonic_reference(12, 250, Ns, Nh=1)

# X = X.T
# Y = Y.T

print(X.shape(), Y.shape())

(4, 50) (2, 50)


In [ ]:
Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)
M = np.dot(M1, M2)

In [ ]:
lam, V = solve_qr(M, iterations=100)
print(np.sqrt(lam))

array([0.2094223883303393, 0.09026236148649224, 0.05508118227979756, 0.004485854551801153], dtype=float64)


In [ ]:
import ujson as json

with open('data.json', 'w') as jsonfile:
    json.dump(X, jsonfile)

In [ ]:
import ujson as json

with open('xy.json') as f:
    data = json.load(f)

In [ ]:
X = np.array(data['X'])
Y = np.array(data['Y'])

In [117]:
from lib.computation import solve_eig_qr

lam, V = solve_eig_qr(np.dot(A, A.transpose()), 2)
print(lam, V[:, 0])

array([3090257647770.855, 1.84513789676072, 0.690618760063498, -6.033914926509784e-05], dtype=float64) array([-0.02983780014940622, -0.7932272635549372, 0.5364013569551137, 0.2866597256192484], dtype=float64)


In [105]:
Q, R = np.linalg.qr(A)
print(R)

array([[-883550.4911050206, -877556.5360877957, -878760.5076758521, -875939.8801019968],
       [-1.844490843821283e-11, -1.540692204816151, -1.705187238642372, -1.081242480290623],
       [-2.640966340204573e-11, 0.0, 0.4207967891230313, -0.778845463474262],
       [-5.666316844096295e-11, -0.0, 0.0, 1.354608353265618e-08]], dtype=float64)


In [45]:
X = np.dot(A, A.transpose())
print(np.dot(X, V[0]))
print(V[0]*lam[0])

array([-2.643192e+10, -7.02683e+11, 4.751729e+11, 2.539385e+11], dtype=float32)
array([-26136.41, -650175.6, 644677.9, 3.090257e+12], dtype=float32)


In [118]:
#TODO: uLab QR seems to give incorrect result for this matrix
A_null = np.array([[-3.090257e+12, -7.008567e+05,  4.739371e+05,  2.532784e+05],
       [-2.618467e+04, -3.090258e+12,  4.707228e+05,  2.515600e+05],
       [-2.622055e+04, -6.970566e+05, -3.090256e+12,  2.519048e+05],
       [-2.613640e+04, -6.948194e+05,  4.698551e+05, -3.090257e+12]])

In [119]:
Q, R = np.linalg.qr(A_null, mode="reduced")
print(R)

array([[3090257000000.001, 727041.3902643195, -447716.5664473673, -227142.0042689324],
       [-1.04984705117826e-25, 3090258000000.152, 226333.2414550781, 443259.1143798828],
       [2.568302040251067e-12, -0.0, -3090256000000.068, 721760.0402832033],
       [-2.576570284484911e-12, 0.0, -0.000244140625, -3090256999999.907]], dtype=float64)


In [129]:
from lib.decoding import harmonic_reference, CCA
from lib.computation import solve_gen_eig_prob
from ulab import numpy as np

gc.collect()

f0 = 7
stim_freqs = [7, 10, 12]
fs = 200

X_test = synth_X(10, 4, 10)

cca = CCA(stim_freqs, fs)

print(cca.compute_corr(X_test))


{12: nan, 10: nan, 7: nan}


In [144]:
print(Y)

array([[-2.171207463750402, -1.338522784901037, -0.5880826741808868, 0.04396681273567864, 0.5271820793288486, 0.8382883489982348, 0.9623007290342446, 0.8932459800949776, 0.6344502260963639, 0.1983787465439827],
       [1.305341814301688, 1.170640910201912, 0.9533601308922323, 0.6639651258411464, 0.3163950326494211, -0.07260892208832805, -0.484309788674666, -0.8988773871495461, -1.296343457179863, -1.657563458793997],
       [0.267963264843056, 0.7796799141142366, 1.073691207707363, 1.094033302783363, 0.8368341650474374, 0.3510505945826392, -0.2708505000816015, -0.9104927984376731, -1.446123014681661, -1.77578613587716],
       [1.850696116855231, 1.432374314847385, 0.8242438445293502, 0.142059844591912, -0.4843267608957941, -0.9356858528536302, -1.12610308092582, -1.019333307398602, -0.6356997203802178, -0.04822539836981504]], dtype=float64)


In [138]:
from lib.decoding import harmonic_reference

X = X_test
Y = harmonic_reference(7, 200, np.max(X_test.shape()), Nh=2, standardise_out=True)

Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)

lam, _ = max_eig(np.dot(M1, M2), 20)

In [151]:
print(np.linalg.inv(Cxx))

array([[597636288720971.6, -590244015165738.0, -70701016851736.37, 63307345255390.93],
       [-387237973120798.1, 382448155728474.1, 45810658901710.62, -41019935647656.07],
       [-46384381598032.41, 45810634090902.22, 5487518240656.996, -4913662265777.822],
       [41533661101626.68, -41019913615534.64, -4913662287824.636, 4399817679136.317]], dtype=float64)


In [20]:
def zeros_like(A):
    return np.zeros(A.shape)

def block_diag(X, Y, reverse=False):
    if not reverse:
        X = np.concatenate((X, zeros_like(X)), axis=1)
        Y = np.concatenate((zeros_like(Y), Y), axis=1)
    else:
        X = np.concatenate((zeros_like(X), X), axis=1)
        Y = np.concatenate((Y, zeros_like(Y)), axis=1)
    return np.concatenate((X, Y), axis=0)

X = np.array([[0.0, 0.2552531, 0.4935954, 0.6992362, 0.8585516, 0.9609866, 0.9997549, 0.972288, 0.8804055, 0.7301948],
       [0.0, 0.2651061, 0.5112409, 0.7207904, 0.8787591, 0.9738424, 0.9992362, 0.9531231, 0.838803, 0.6644569],
       [0.0, 0.2634635, 0.5083104, 0.7172394, 0.8754874, 0.9718722, 0.9995833, 0.9566626, 0.8461428, 0.6758333],
       [0.0, 0.2671577, 0.5148946, 0.7252015, 0.8827904, 0.9762053, 0.9986557, 0.9485094, 0.8294118, 0.6500207]])

# Y = cca_reference([7], 200, 10, Nh=2)

Y = np.array([[-2.171207, -1.338523, -0.5880827, 0.04396701, 0.5271821, 0.8382883, 0.9623005, 0.8932458, 0.6344502, 0.1983788],
       [1.305342, 1.170641, 0.9533603, 0.6639652, 0.3163952, -0.07260892, -0.4843098, -0.8988775, -1.296343, -1.657563],
       [0.2679634, 0.7796801, 1.073691, 1.094033, 0.8368343, 0.3510505, -0.2708505, -0.9104931, -1.446123, -1.775786],
       [1.850696, 1.432374, 0.8242437, 0.1420597, -0.4843266, -0.9356859, -1.126103, -1.019333, -0.6356997, -0.04822552]])

In [38]:
from lib.computation import solve_gen_eig_prob

Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

A = block_diag(Cxy, Cyx, reverse=True)
B = block_diag(Cxx, Cyy)

lam, Phi = solve_gen_eig_prob(A, B, eps=1e-6)

Traceback (most recent call last):
  File "<stdin>", line 11, in <module>
  File "computation.py", line 23, in solve_gen_eig_prob
ValueError: input matrix is asymmetric


In [41]:
from lib.computation import solve_eig_qr

print(A)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ImportError: can't import name solve_eig_qr


In [37]:
eps = 1e-6

def replace_nan(A, rep=0):
    return np.where(np.isfinite(A), A, rep)

Lam_b, Phi_b = np.linalg.eig(B) # eig decomp of B alone
Lam_b = np.eye(len(Lam_b))*Lam_b # convert to diagonal matrix of eig vals

Lam_b_sq = replace_nan(Lam_b**0.5)+np.eye(len(Lam_b))*eps
print(np.linalg.inv(Lam_b_sq))

array([[0.2104852, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1267.649, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 12.49017, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 1000000.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 8.926649, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.873826, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3365085, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1799781]], dtype=float32)


In [7]:
import urandom

print([urandom.random() for i in range(10)])


[0.00769949, 0.5943487, 0.2485597, 0.6857158, 0.00562346, 0.3077182, 0.6192346, 0.6785934, 0.7380899, 0.1414928]


In [109]:
%ls

Listing directory '/'.
     3117    .env
      139    boot.py
      487    data.json
             lib/


In [50]:
%ls --recursive lib

Listing directory 'lib'.
       27    lib/__init__.mpy
     1643    lib/computation.mpy
      558    lib/config.mpy
     1065    lib/core.mpy
      722    lib/decoding.mpy
     8703    lib/mqtt_as.mpy
      790    lib/networking.mpy
     2504    lib/peripherals.mpy
     1052    lib/scheduling.mpy
     1174    lib/signal.mpy
     2601    lib/umqtt.mpy
      614    lib/utils.mpy
      641    lib/websockets.mpy


In [35]:
%sendtofile --source lib/.env .env  --binary

Sent 3116 bytes in 104 chunks to .env.


In [195]:
%sendtofile lib/core.py --source lib/core.py

Sent 83 lines (2491 bytes) to lib/core.py.


In [131]:
%sendtofile lib/decoding.py --source lib/decoding.py

Sent 61 lines (2106 bytes) to lib/decoding.py.


In [191]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [111]:
%fetchfile data.json --print

Fetched 487=487 bytes from data.json.


In [12]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US

In [ ]:
# see https://docs.micropython.org/en/latest/reference/constrained.html

import gc
import micropython

gc.enable()

gc.collect()
micropython.mem_info()
print('-----------------------------')
print('Initial free: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
def func():
    return
    with open('xy.json') as f:
        data = json.load(f)
        del data
gc.collect()
print('Func definition: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
func()
print('Func run free: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
gc.collect()
print('Garbage collect free: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
print('-----------------------------')
# micropython.mem_info(1)